<a href="https://colab.research.google.com/github/joseph7104/-1INF46-Plan_Compras_Produccion/blob/master/notebooks/preprocesamiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/joseph7104/-1INF46-Plan_Compras_Produccion.git
%cd "./-1INF46-Plan_Compras_Produccion"

Cloning into '-1INF46-Plan_Compras_Produccion'...
remote: Enumerating objects: 272, done.
remote: Counting objects: 100% (272/272), done.
remote: Compressing objects: 100% (219/219), done.
remote: Total 272 (delta 106), reused 172 (delta 48), pack-reused 0 (from 0)
Receiving objects: 100% (272/272), 20.11 MiB | 24.46 MiB/s, done.
Resolving deltas: 100% (106/106), done.
/content/-1INF46-Plan_Compras_Produccion


In [2]:
import os
RAW_DIR  = "data/raw"
PROC_DIR = "data/processed"
#Verificamos la existencia de los archivos
print("RAW:", os.listdir(RAW_DIR))


RAW: ['calendario_peru.csv', 'linea.csv', 'produccion.csv', 'hoja_ruta.csv', 'work_center_calendar.csv', 'work_center.csv', 'plato.csv', 'planificacion_detalle_insumo.csv', 'compras.csv', 'proveedor.csv', 'receta.csv', 'ignore', 'insumo.csv', 'ventas.csv', 'almacen.csv', 'planificacion.csv', 'rccp_standard.csv', 'inventario_movimientos.csv', 'inventario_diario.csv', 'almacen_ubicacion.csv', 'proveedor_x_insumo.csv']


In [3]:
import pandas as pd
import numpy as np

# rutas (las tuyas)
VENTAS_CSV      = f"{RAW_DIR}/ventas.csv"
CALENDARIO_CSV  = f"{RAW_DIR}/calendario_peru.csv"

# --- Carga ---
ventas = pd.read_csv(VENTAS_CSV, parse_dates=['fecha'], dayfirst=True, keep_default_na=False)
cal    = pd.read_csv(CALENDARIO_CSV, parse_dates=['fecha'], dayfirst=True, keep_default_na=False)

# normaliza fechas (robusto)
ventas['fecha'] = pd.to_datetime(ventas['fecha'], errors='coerce', format='%Y-%m-%d')
cal['fecha']    = pd.to_datetime(cal['fecha'],    errors='coerce', format='%Y-%m-%d')

# --- Normaliza nombres desde tu plantilla ---
ventas = ventas.rename(columns={
    'id_plato':'plato', # Based on the previous error and user input, it seems 'id_plato' should be renamed to 'plato'
    'cantidad':'cantidad',
    'precio_total':'monto_total'
})
assert {'fecha','plato','cantidad','monto_total'}.issubset(ventas.columns), "Revisa columnas en VENTAS.csv"

# --- Agregación diaria por plato ---
agg_dict = {
    'cantidad':    ('cantidad', 'sum'),
    'monto_total': ('monto_total', 'sum')
}

# Si ventas ya trae la columna 'temporada' desde el Data Generator
if 'temporada' in ventas.columns:
    agg_dict['temporada'] = ('temporada', 'first')

ventas_dia = (ventas.groupby(['fecha','plato'], as_index=False)
                     .agg(**agg_dict))

#ventas_dia = (ventas.groupby(['fecha','plato'], as_index=False)
#                     .agg(cantidad=('cantidad','sum'),
#                          monto_total=('monto_total','sum'))
#                     .rename(columns={'fecha':'fecha'}))

# --- Calendario: solo columnas útiles ---
use_cols = ['fecha'] + [c for c in ['feriado','evento','clima'] if c in cal.columns]
cal = cal[use_cols].drop_duplicates('fecha')

# --- Merge ---
df = ventas_dia.merge(cal, on='fecha', how='left')

# --- Features de fecha ---
df['anio'] = df['fecha'].dt.year
df['mes']  = df['fecha'].dt.month
df['dia']  = df['fecha'].dt.day
df['dow']  = df['fecha'].dt.dayofweek           # 0=lun … 6=dom
df['fin_de_semana'] = (df['dow']>=5).astype(int)

# --- Dummies de TODOS los eventos (dinámico) ---
if 'evento' in df.columns:
    eventos_unicos = (
        df['evento']
        .astype(str)
        .str.strip()
        .replace({'': np.nan, 'nan': np.nan})
        .dropna()
        .unique()
    )

    # crea una dummy por cada evento único
    for ev in eventos_unicos:
        clean_ev = (
            ev.strip()
              .replace(" ", "_")
              .replace(".", "")
              .replace(",", "")
              .replace("ñ","n")
              .replace("á","a").replace("é","e")
              .replace("í","i").replace("ó","o").replace("ú","u")
        )
        df[f'evento_{clean_ev}'] = df['evento'].str.fullmatch(ev, case=False).astype('Int64')

    # (opcional) guarda el listado de eventos detectados
    try:
        pd.DataFrame({'evento_unico': eventos_unicos}).to_csv(f"{PROC_DIR}/eventos_unicos.csv", index=False)
    except Exception as e:
        print("Aviso: no se pudo guardar eventos_unicos.csv:", e)

# --- Dummies de temporada ---
if 'temporada' in df.columns:
    df = pd.get_dummies(df, columns=['temporada'], prefix='temp', dummy_na=False)

# --- Dummies de clima (si existe) ---
if 'clima' in df.columns:
    df = pd.get_dummies(df, columns=['clima'], prefix='clima', dummy_na=False)

# --- Orden de columnas (manteniendo tus nombres) ---
base = ['fecha','plato','cantidad','monto_total','anio','mes','dia','dow','fin_de_semana']
extra = [c for c in df.columns if c.startswith('evento_') or c.startswith('clima_') or c.startswith('temp_') or c=='feriado']
df = df[base + extra].sort_values(['plato','fecha']).reset_index(drop=True)

# --- Guardar ---
#out_daily = f"{PROC_DIR}/dataset_forecast_diario.csv"
#df.to_csv(out_daily, index=False)
print("✅ Guardado — filas:", len(df))
df.head()

✅ Guardado — filas: 21698


,fecha,plato,cantidad,monto_total,anio,mes,dia,dow,fin_de_semana,feriado,...,evento_Navidad,evento_Dia_del_Padre,evento_Dia_del_Ceviche,temp_invierno,temp_otono,temp_primavera,temp_verano,clima_Lluvioso,clima_Nublado,clima_Soleado
0,2021-01-01,1,30.0,780.0,2021,1,1,4,0,1,...,0,0,0,False,False,False,True,True,False,False
1,2021-01-02,1,18.0,468.0,2021,1,2,5,1,0,...,0,0,0,False,False,False,True,False,False,True
2,2021-01-03,1,25.0,650.0,2021,1,3,6,1,0,...,0,0,0,False,False,False,True,False,False,True
3,2021-01-04,1,20.0,520.0,2021,1,4,0,0,0,...,0,0,0,False,False,False,True,True,False,False
4,2021-01-05,1,20.0,520.0,2021,1,5,1,0,0,...,0,0,0,False,False,False,True,False,True,False


In [4]:
num_rows, num_cols = df.shape
print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_cols}")

Number of rows: 21698
Number of columns: 32


In [5]:
# Asegura orden correcto
df = df.sort_values(['plato','fecha']).reset_index(drop=True)

# ---------- LAGS por plato ----------
# Usa ventas pasadas como predictores
lags = [1, 7, 14, 28]
for L in lags:
    df[f'lag_{L}'] = df.groupby('plato')['cantidad'].shift(L)

# ---------- ROLLING STATS por plato (sin fuga de info) ----------
# Usamos la serie desplazada 1 día para que el rolling no mire el día actual
def add_rolling(group, windows=(7,14,28)):
    s = group['cantidad'].shift(1)
    out = {}
    for w in windows:
        out[f'rolling_mean_{w}'] = s.rolling(w).mean()
        out[f'rolling_std_{w}']  = s.rolling(w).std()
    return pd.DataFrame(out, index=group.index)

roll_feats = df.groupby('plato', group_keys=False).apply(add_rolling)
df = pd.concat([df, roll_feats], axis=1)

# ---------- (Opcional) codificación cíclica de calendario ----------
# Mejora modelos lineales; para árboles no es obligatorio
if 'dow' in df.columns:
    df['dow_sin'] = np.sin(2*np.pi*df['dow']/7)
    df['dow_cos'] = np.cos(2*np.pi*df['dow']/7)
if 'mes' in df.columns:
    df['mes_sin'] = np.sin(2*np.pi*(df['mes']-1)/12)
    df['mes_cos'] = np.cos(2*np.pi*(df['mes']-1)/12)

# ---------- Dataset para modelar (quitar filas con NaN iniciales de lags/rolling) ----------
min_required = ['lag_1','lag_7','lag_14','rolling_mean_7','rolling_std_7']
df_model = df.dropna(subset=[c for c in min_required if c in df.columns]).reset_index(drop=True)

# Guardar (ajusta PROC_DIR si usas otra ruta)
out_features = f"{PROC_DIR}/dataset_forecast_diario.csv"
df_model.to_csv(out_features, index=False)
print("✅ Features añadidas y guardadas en:", out_features)
print("Filas totales:", len(df), "→ filas para modelar (sin NaN de arranque):", len(df_model))


/tmp/ipython-input-2255251862.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  roll_feats = df.groupby('plato', group_keys=False).apply(add_rolling)


✅ Features añadidas y guardadas en: data/processed/dataset_forecast_diario.csv
Filas totales: 21698 → filas para modelar (sin NaN de arranque): 21530


In [6]:
!git status


On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   data/processed/dataset_forecast_diario.csv

no changes added to commit (use "git add" and/or "git commit -a")


In [7]:
!git add .

In [11]:
!git commit -m "forecast_input"

[master cd2a524] forecast_input
 1 file changed, 21531 insertions(+), 21553 deletions(-)


In [9]:
!git config --global user.email "joseph7104@gmail.com"
!git config --global user.name "joseph7104"

In [12]:
# Importar la librería para acceder a los secrets
from google.colab import userdata
import os

# --- Reemplaza estos valores con los tuyos ---
username = "joseph7104"
repository = "-1INF46-Plan_Compras_Produccion"  # <-- Cambia esto por el nombre de tu repositorio
# ---------------------------------------------

# Obtener el token guardado en los secrets de Colab
token = userdata.get('TOKEN')

# Construir la URL del repositorio con el token de autenticación
# El formato es: https://<token>@github.com/<usuario>/<repositorio>.git
repo_url_with_token = f"https://{token}@github.com/{username}/{repository}.git"

# Empujar los cambios al repositorio remoto usando la URL con el token
# Primero, eliminamos el 'origin' viejo para evitar conflictos
!git remote remove origin
# Luego, añadimos el nuevo 'origin' con el token
!git remote add origin {repo_url_with_token}
# Finalmente, hacemos el push
!git push origin master

Enumerating objects: 9, done.
Counting objects: 100% (9/9), done.
Delta compression using up to 2 threads
Compressing objects: 100% (5/5), done.
Writing objects: 100% (5/5), 1.05 MiB | 2.89 MiB/s, done.
Total 5 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/joseph7104/-1INF46-Plan_Compras_Produccion.git
   a5fd243..cd2a524  master -> master
